# A very quick introduction to Automatic Differention and Gradient Descent

## Example: using Gradient Descent to solve a Linear Regression Problem

## IADS Summer School, 2nd August 2022

### Dr Michael Fairbank, University of Essex, UK

- Email: m.fairbank@essex.ac.uk
- This is Jupyter Notebook 1.1 of the course

## Build a scatter diagram

- First build some datapoints for a simple linear regression example...

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

# dataset for a simple regression problem (1 input 1 output):
dataset_size=10
noise_level=1
np.random.seed(1) # Seed the random number generator so the whole class have consistent results
x_train=(np.random.rand(dataset_size,1)*5+2).astype(np.float32)
y_train=(3*x_train+1+np.random.normal(scale=noise_level, size=(dataset_size,1))).astype(np.float32)

In [ ]:
print("x_train",x_train.shape, x_train)
print("y_train",y_train.shape, y_train)

In [ ]:
# show training set
plt.scatter(x_train, y_train)

## Build a linear regression model
- We will build a linear regression model $y=wx+b$ to try to fit the data.  
- We don't know what $w$ and $b$ are yet though.

In [ ]:
import tensorflow as tf

w=tf.Variable(0.0, tf.float32)
b=tf.Variable(0.0, tf.float32)

def model(x):
    return x*w+b
print(model(x_train))

Let's show how our model depends upon $w$:

In [ ]:
w.assign(0.1)
print(model(x_train))

- In the cell below, try to print what the model outputs given input x=0.65.
    - **Try This** (Check: we should get 0.065)
    
- Remember in Jupyter Notebooks you can use "menu:cell:run cells" to run the currently highlighted cells.
        - the same can be achieved with ctrl+Enter
    - Also in Jupyter, in some notebooks you might need to "Menu:Cell:run all" to refresh all variables in the correct order
        - or even use "Kernel: Restart and run all"

In [ ]:
# TODO put some code here


- **Try This**: Change b to 0.2 and evaluate the model given input x=0.65

In [ ]:
# TODO put some code here


### Define a "loss function" 
- To apply gradient descent, we need to define a loss function.
- This loss function calculates how well the model is doing
- We will manually encode the mean-squared error

In [ ]:
def loss_function(x,y):
    return tf.reduce_mean(tf.square(model(x)-y))
print(loss_function(x_train, y_train))

- **Try this:**
    - In the cell below, evaluate the loss function when x=1 and the output is supposedly 4.
    - Compare this to the actual model output when x=1

In [ ]:
# TODO put some code here


### Use Automatic Differentiation

- Automatic differention is one of the amazing things about tensorflow, which allows Keras to build and train neural networks very easily.  
- It uses "backpropagation" to compute the necessary gradients.
- Here we'll use it to compute the gradient of the loss function with respect to $w$ and $b$

In [ ]:
with tf.GradientTape() as tape:

    # Run the forward pass of the layer.
    # The operations that the layer applies
    # to its inputs are going to be recorded
    # on the GradientTape.
    L=loss_function(x_train, y_train)
print("L",L.numpy())
grads = tape.gradient(L, [w,b])
print("w,b",w.numpy(),b.numpy())
print("dL/dw",grads[0].numpy())
print("dL/db",grads[1].numpy())


### Apply an iteration of "gradient descent"
If we aim to improve $L$ a tiny bit then we can use the following two updates:
- $w \leftarrow w-\eta \frac{\partial L}{\partial w}$
- $b \leftarrow b-\eta \frac{\partial L}{\partial b}$

where $\eta>0$ is any small number we choose.

This is what is meant by "gradient descent".

In [ ]:
eta=0.001
w.assign(w-eta*grads[0])
b.assign(b-eta*grads[1])
print("w,b",w.numpy(),b.numpy())
L=loss_function(x_train, y_train)
print("L",L.numpy())

### Iteratively apply gradient descent

- Run the code block below serveral times and see what $w$ and $b$ converge to.  
    - In Jupyter, Click ctrl+enter to run a block.
- **Extra Challenge:** Try to wrap the code block below into a python "for loop" to run 10000 times.

In [ ]:
with tf.GradientTape() as tape:
    L=loss_function(x_train, y_train)
grads = tape.gradient(L, [w,b])

eta=0.001
w.assign(w-eta*grads[0])
b.assign(b-eta*grads[1])
print("w,b",w.numpy(),b.numpy())
new_L=tf.reduce_mean(tf.square(model(x_train)-y_train))
print("new_L",new_L.numpy(),". Old L",L.numpy())

###  Use a built-in optimizer

- Instead of hand coding the updates for $w$ and $b$, we can use a built-in optimiser.
- For ordinary gradient descent we use "SGD" optimizer.


In [ ]:
import tensorflow.keras as keras
w_history=[] # keep a history for plotting purposes
b_history=[]
L_history=[]
w.assign(0) # We'll restart from scratch in this script
b.assign(0)

In [ ]:
eta=0.001
optimizer=keras.optimizers.SGD(eta)
for i in range(1000):
    with tf.GradientTape() as tape:
        L=loss_function(x_train, y_train)
        
    grads = tape.gradient(L, [w,b])
    optimizer.apply_gradients(zip(grads, [w,b])) # This line updates w and b with 1 step of gradient descent
    
    w_history.append(w.numpy()) # append to history for plotting purposes
    b_history.append(b.numpy())
    L_history.append(L.numpy())
print("w,b",w.numpy(),b.numpy())
new_L=tf.reduce_mean(tf.square(model(x_train)-y_train))
print("new_L",new_L.numpy(),". Old L",L.numpy())

### Compare to optimal values for linear regression

In [ ]:
import scipy.stats as stats
slope, intercept, r_value, p_value, std_err = stats.linregress(x_train.reshape(-1),y_train.reshape(-1))
print("slope",slope)
print("intercept",intercept)

Plot the history graphs to see how well the variables converged.

In [ ]:
fig=plt.figure(figsize=[15, 3])
ax1=fig.add_subplot(1,3, 1)
ax1.plot(L_history)
ax1.set_title("L history")              
ax2=fig.add_subplot(1,3, 2)
ax2.plot(w_history)
ax2.set_title("w history")              
ax3=fig.add_subplot(1,3, 3)
ax3.plot(b_history)
ax3.set_title("b history")              

**Exercise:**

1. Rerun the gradient descent iterations until the model's $w$ and $b$ parameters match the actual slope and intercept values calculated by stats.linregress
2. Increase / decrease the learning rate as you see fit.  

**Questions:**
1. What happens if eta is too large?  **TODO:** Write answer here
2. What happens if eta is too small?  **TODO:** Write answer here

## Using a Keras "model" to hold $w$ and $b$
Instead of building our own linear model $y=wx+b$ as above, we can use the in-built keras "Dense" layer

- We will use a keras "Dense" layer with 1 input and 1 output.
- This keras dense layer represents a model $y=wx+b$, where $w$ and $b$ are learnable variables.  
    - $w$ is called the "kernel", and $b$ is called the "bias"
    - Note that $w$ is a 1*1 matrix, and $b$ is a length-1 vector.

In [ ]:
from tensorflow.keras import layers

class LinearModel(keras.Model):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.layer1=layers.Dense(1)
    def call(self, x):
        return self.layer1(x)
    
keras_linear_model=LinearModel()
keras_linear_model(x_train)
print("This is the w value",keras_linear_model.layer1.kernel.numpy())
print("This is the b value",keras_linear_model.layer1.bias.numpy())


## Use Keras "fit" for the training loop
- Keras allows us to quickly define the optimizer to use (SGD), and the loss function to use
- It also provides a "fit" function which executes the training loop.

Run and study the following code.

In [ ]:
keras_linear_model.compile(
    optimizer=keras.optimizers.SGD(0.01),  # Optimizer
    loss=keras.losses.MeanSquaredError()
)
history = keras_linear_model.fit(
    x_train,
    y_train,
    batch_size=len(x_train),
    epochs=10000,
    verbose=0
)

In [ ]:
print("w",keras_linear_model.layer1.kernel.numpy(),"Optimal w:",slope)
print("b",keras_linear_model.layer1.bias.numpy(),"Optimal b:",intercept)


## Summary

- We have built this simple model:
<img src="./images/ffnn_single_layer.svg" alt="1-layer FFNN" width="300">
- We've been introduced to Automatic Differntiation, and gradient descent.
- We've also seen the keras Dense layer, the Keras loss functions, optimizer and fit loop

- We'll study all of these more in the next workbook and beyond...